In [1]:
import os
import sys
import ee
import geemap
import json
from google.oauth2.service_account import Credentials
import geopandas as gpd


In [2]:
# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)

In [3]:
# Import SHAPEFILE_PATH from constants
from constants import STUDY_BOUNDARY_PATH

In [4]:
'''# Get the path to the service account JSON file from the environment variable
key_content  = os.environ.get('EEPRIVATE_KEY')
'''

"# Get the path to the service account JSON file from the environment variable\nkey_content  = os.environ.get('EEPRIVATE_KEY')\n"

In [5]:

'''# Parse the JSON content
key_dict = json.loads(key_content)
# Create credentials from the parsed JSON content
SCOPES = [
    'https://www.googleapis.com/auth/earthengine',
    'https://www.googleapis.com/auth/cloud-platform'
]
credentials = Credentials.from_service_account_info(key_dict, scopes=SCOPES)'''

#ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [6]:
imagery = ee.Image("projects/mapbiomas-chaco/public/collection4/mapbiomas_chaco_collection4_integration_v1")


In [7]:
imagery.bandNames()

In [58]:
chaco_2013 = imagery.select('classification_2013')
Map = geemap.Map()
Map.addLayer(chaco_2013,{}, 'Mapbiomas')
Map.center_object(chaco_2013)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
shapefile_path = STUDY_BOUNDARY_PATH
study_boundary = gpd.read_file(shapefile_path)

In [10]:
ee_boundary = geemap.geopandas_to_ee(study_boundary)



In [59]:

clipped_dataset = chaco_2013.clip(ee_boundary)


In [60]:
Map2 = geemap.Map()

centroid = ee_boundary.geometry().centroid().getInfo()['coordinates']
Map2.setCenter(centroid[0], centroid[1], 7)

Map2.addLayer(clipped_dataset, {}, 'MB Dataset Clipped')
Map2

Map(center=[-21.999008038350066, -59.79579144156555], controls=(WidgetControl(options=['position', 'transparen…

In [30]:
clipped_dataset

In [13]:
# Extract the first feature from the FeatureCollection
ee_boundary_feature = ee_boundary.first()

# Get the geometry of the feature
ee_boundary_geometry = ee_boundary_feature.geometry()

# Get the coordinates of the geometry
ee_boundary_coordinates = ee_boundary_geometry.coordinates().getInfo()

In [61]:
export_params = {
    'scale': 30, # Resolution in meters
    'region': ee_boundary_coordinates, # Export only the region of interest
    'crs': 'EPSG:4326', # Coordinate reference system (optional)
    'fileFormat': 'GeoTIFF', # Export format (GeoTIFF or other supported formats)
    'fileNamePrefix': 'clipped_dataset13', # Prefix for the exported file name
}
# Get the geometry and scale (resolution) of the clipped raster
geometry = ee_boundary.geometry()
scale = export_params['scale']

# Compute the pixel dimensions of the exported raster
dimensions = ee.Image.pixelLonLat().reproject(geometry.projection().atScale(scale)).reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=geometry,
    scale=scale,
    maxPixels=1e10
).getInfo()

pixel_width = dimensions['longitude_max'] - dimensions['longitude_min']
pixel_height = dimensions['latitude_max'] - dimensions['latitude_min']


In [62]:
# Your existing export task code
export_task = ee.batch.Export.image.toDrive(
    image=clipped_dataset.select('classification_2013'),
    description='classification_2013',
    folder='mb_data', # specify a folder in your Google Drive
    maxPixels=1e10,
    **export_params
)

export_task.start()

 https://code.earthengine.google.com/tasks

In [38]:
print(export_task.status())


{'state': 'COMPLETED', 'description': 'classification_2022', 'creation_timestamp_ms': 1694661857778, 'update_timestamp_ms': 1694662357885, 'start_timestamp_ms': 1694661873536, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1W3tqm-V-aPZIkhUchu9x_uEH9znW0lOO'], 'attempt': 1, 'batch_eecu_usage_seconds': 3112.013427734375, 'id': 'I76PYE3NHIKHCHQ3U632YDYZ', 'name': 'projects/earthengine-legacy/operations/I76PYE3NHIKHCHQ3U632YDYZ'}
